In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

In [ ]:
files.upload()


In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d saurabhshahane/barkvn50

 88% 154M/176M [00:00<00:00, 303MB/s]
100% 176M/176M [00:00<00:00, 298MB/s]


In [8]:
! mkdir train

In [ ]:
! unzip barkvn50.zip -d train

In [10]:
# import statements
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [11]:
#Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.05,
        height_shift_range=0.05,
        fill_mode="nearest",
        validation_split=0.1,
        horizontal_flip=True)

src_path='/content/train/BarkVN-50/BarkVN-50_mendeley'

In [12]:
#defining datasets
batch_size = 64
train_generator = train_datagen.flow_from_directory(
    directory=src_path,
    target_size=(303, 404),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_directory(
    directory=src_path,
    target_size=(303, 404),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)

Found 5039 images belonging to 50 classes.
Found 539 images belonging to 50 classes.


In [13]:
#CNN Model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=[303, 404, 3]), 
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'), MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    Conv2D(256, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(50, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 301, 402, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 201, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 148, 199, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 72, 97, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 70, 95, 64)        3

In [ ]:
#Training the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x=train_generator, validation_data=valid_generator, epochs=50)

Epoch 1/50
79/79 [==============================] - 152s 2s/step - loss: 3.8942 - accuracy: 0.0383 - val_loss: 3.7471 - val_accuracy: 0.0557
Epoch 2/50
79/79 [==============================] - 138s 2s/step - loss: 3.6905 - accuracy: 0.0718 - val_loss: 3.5591 - val_accuracy: 0.0686
Epoch 3/50
79/79 [==============================] - 137s 2s/step - loss: 3.4090 - accuracy: 0.1006 - val_loss: 3.2407 - val_accuracy: 0.1039
Epoch 4/50
79/79 [==============================] - 136s 2s/step - loss: 3.2049 - accuracy: 0.1274 - val_loss: 3.1015 - val_accuracy: 0.1596
Epoch 5/50
79/79 [==============================] - 136s 2s/step - loss: 3.0757 - accuracy: 0.1627 - val_loss: 3.0360 - val_accuracy: 0.1855
Epoch 6/50
79/79 [==============================] - 135s 2s/step - loss: 2.8552 - accuracy: 0.2133 - val_loss: 3.1083 - val_accuracy: 0.1596
Epoch 7/50
79/79 [==============================] - 136s 2s/step - loss: 2.6405 - accuracy: 0.2550 - val_loss: 2.6398 - val_accuracy: 0.2839
Epoch 8/50
79